In [ ]:
import os
import shutil
from teemof.initialize import *
from teemof.run import initialize_trial

## Select sample files to modify

In [ ]:
# sample_files['data'] = os.path.join(os.getcwd(), 'sample', 'lammps.data.sample')
# sample_files['inp'] = os.path.join(os.getcwd(), 'sample', 'in3.cond.sample')

inp_lines, data_lines, qsub_lines = get_files(sample_files)
[print(i, sample_files[i]) for i in sample_files]

## Trial directory

In [ ]:
trial_name = 'Trial6B'

trial_dir = os.path.join(os.getcwd(), 'Trials', trial_name)
os.makedirs(trial_dir) if not os.path.exists(trial_dir) else None

## Single Trial

In [ ]:
coefficient_list = [[1, 1, 0, 5], [2, 2, 0, 5], [1, 2, 0.1, 5]]
thermo_step = 100
masses = [[1, 39.948], [2, 20.1797]]

inp_lines = change_thermo(inp_lines, thermo=thermo_step)
inp_lines = change_pair_coeff(inp_lines, coefficient_list)
data_lines = change_masses(data_lines, masses)
qsub_lines = lammps_qsub(qsub_lines, name='TrialX', walltime='12:00:00', nodes=1, ppn=4, queue='shared')

number_of_runs = 10
initialize_trial(trial_dir, number_of_runs, inp_lines, data_lines, qsub_lines)

## Multiple Trials

In [ ]:
# sigma = [4, 4.5, 4.75, 5.0, 5.25, 5.5, 6.0]
sigma = [3.0, 3.25, 3.5, 3.75, 4.25]
eps = 0.1
number_of_runs = 5

for trial_index, sg in enumerate(sigma):
    new_trial_dir = os.path.join(trial_dir, '%s-%i' % (trial_name, trial_index + 1))      # Initialize trial directory
    if not os.path.exists(new_trial_dir):
        os.mkdir(new_trial_dir)
        
    coefficient_list = [[1, 1, 0, sg],
                        [2, 2, 0, sg],
                        [1, 2, eps, sg]]

    input_lines = change_thermo(inp_lines, thermo=100)
    input_lines = change_pair_coeff(inp_lines, coefficient_list)
    
    run_info = dict(sigma=sg, epsilon=eps, id="Sigma: %.2f" % sg, thermo=100, files=sample_files)
    initialize_trial(new_trial_dir, number_of_runs, input_lines, data_lines, qsub_lines, info=run_info)